<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/HOML_Exercise_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

#8.

# Grammer_StringGenerater

In [3]:
#문법에 맞는 문자열을 반환하는 함수

#문법, LR table 과 비슷함
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    #마지막
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    #마지막
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output=[]
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        #다음 문자와 상태
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [4]:
for _ in range(25):
    #25개의 문자열 생성
    print(generate_string(default_reber_grammar), end=" ")

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE 

#Wrong_StringGenrater

In [5]:
#모든 가능한 알파벳들
POSSIBLE_CHARS = "BEPSTVX"

#잘못된 문자열 만들기
def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    #올바른 문자열에
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    #있을수는 있지만 올바른 문자열엔 없는 문자열을
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    #중간에 하나 추가함
    return good_string[:index] + bad_char + good_string[index + 1:]

In [6]:
#잘못된 문자열
for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

BTBPVVETV BTBTSSSPXXVVETE BTBTXXTTSTVPSETE BBBTXSETE BPBTPXSEPE BPBPVPXTTTTVPXTVPXVPXTTTVVEVE BTBXXXTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE BPBPVVEPT BTBPTVEETE BTBTSSXXTTVXETE BTBTSXTTVVETE BPBPVVTPE BTBTSXTTVVETE EPBPVPXVVEPE BPTTXSEPE BPBTXXSPXTVVEPE BTBTXSPTE BPTTSXXTVPXVVEPE PPBPVPSEPE 

#Char_To_Int

In [7]:
#문자열을 숫자로 바꾸기, 문자의 인덱스 리스트로 변경한다.
def strings_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

#DataGenerater

In [8]:
#올바른 문자열, 잘못된 문자열 1 : 1 인 샘플과 레이블 생성
def generate_dataset(size):
    good_strings = [strings_to_ids(generate_string(embedded_reber_grammar)) for _ in range(size // 2)]
    bad_strings = [strings_to_ids(generate_corrupted_string(embedded_reber_grammar)) for _ in range(size // 2)]
    all_strings = good_strings + bad_strings
    #ragged 텐서 생성
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] + [[0.] for _ in range(len(bad_strings))])
    return X, y


#Generate_Data

In [9]:
X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)
X_test, y_test = generate_dataset(2000)

In [10]:
print(X_train[0])
s = ''
for i in X_train[0]:
    s += POSSIBLE_CHARS[i]
print(s, y_train[0][0])

tf.Tensor([0 4 0 2 5 5 1 4 1], shape=(9,), dtype=int32)
BTBPVVETE 1.0


#Binary_Sequence_Classifier

In [11]:
embedding_size = 5

model = keras.models.Sequential([
    #인풋데이터는 ragged tensor, int32 타입
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation='sigmoid')
])

#네스테로프 가속 경사하강법 
optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum=0.95, nesterov=True)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#Model_Train

In [13]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
313/313 [==============================] - 18s 58ms/step - loss: 0.5617 - accuracy: 0.6856 - val_loss: 0.5538 - val_accuracy: 0.7045
Epoch 2/20
313/313 [==============================] - 18s 57ms/step - loss: 0.4474 - accuracy: 0.7943 - val_loss: 0.3441 - val_accuracy: 0.8490
Epoch 3/20
313/313 [==============================] - 18s 57ms/step - loss: 0.2955 - accuracy: 0.8861 - val_loss: 0.2465 - val_accuracy: 0.9095
Epoch 4/20
313/313 [==============================] - 18s 57ms/step - loss: 0.2331 - accuracy: 0.9245 - val_loss: 0.3677 - val_accuracy: 0.8675
Epoch 5/20
313/313 [==============================] - 18s 58ms/step - loss: 0.1298 - accuracy: 0.9587 - val_loss: 0.0525 - val_accuracy: 0.9905
Epoch 6/20
313/313 [==============================] - 18s 57ms/step - loss: 0.0239 - accuracy: 0.9944 - val_loss: 0.0054 - val_accuracy: 0.9990
Epoch 7/20
313/313 [==============================] - 18s 57ms/step - loss: 0.0119 - accuracy: 0.9976 - val_loss: 0.0019 - val_accuracy:

#Model_Evaluate

In [14]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 1s 9ms/step - loss: 1.9693e-04 - accuracy: 1.0000


[0.00019692543719429523, 1.0]

#9.

#Random_Date_Genrater

In [26]:
from datetime import date

#January 22, 2019 -> 2019-01-22 형식으로
MONTHS = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    #숫자
    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    #month, day, year 을 빼내기위해서
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    #샘플
    X = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    #타깃 YYYY-mm-dd 형식으로 변환
    y = [dt.isoformat() for dt in dates]
    
    return X, y

In [27]:
n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
April 09, 3947           3947-04-09               
May 07, 7692             7692-05-07               
February 20, 5043        5043-02-20               


#INPUT_OUTPUT_CHARS

In [81]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
OUTPUT_CHARS = "0123456789-"
INPUT_CHARS, OUTPUT_CHARS

38


#Input_Output_Tokenizer

In [194]:
input_chars = MONTHS + ['0123456789, ']
output_chars = ['0123456789-']

input_tokenizer = keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
input_tokenizer.fit_on_texts(sorted(set(words)))
input_vocab_size = len(input_tokenizer.word_index)

output_tokenizer = keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
output_tokenizer.fit_on_texts(sorted(set(output_chars)))
output_vocab_size = len(output_tokenizer.word_index)

In [195]:
print(input_tokenizer.word_index)
print(output_tokenizer.word_index)

{'e': 1, 'r': 2, 'u': 3, 'b': 4, 'a': 5, 'y': 6, 't': 7, 'c': 8, 'm': 9, 'J': 10, 'A': 11, 'p': 12, 'l': 13, 'n': 14, 'M': 15, 'o': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ',': 27, ' ': 28, 'i': 29, 'g': 30, 's': 31, 'D': 32, 'F': 33, 'h': 34, 'N': 35, 'v': 36, 'O': 37, 'S': 38}
{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '-': 11}


In [181]:
X, y = random_dates(3)

print('-'*25)
print(X[0])
X = input_tokenizer.texts_to_sequences(X)
print(X[0])
X = input_tokenizer.sequences_to_texts(X)
print(X[0][::2])

print('-'*25)

print(y[0])
y = output_tokenizer.texts_to_sequences(y)
print(y[0])
y = output_tokenizer.sequences_to_texts(y)
print(y[0][::2])


-------------------------
May 25, 4642
[12, 4, 6, 38, 29, 32, 37, 38, 31, 33, 31, 29]
May 25, 4642
-------------------------
4642-05-25
[5, 7, 5, 3, 11, 1, 6, 11, 3, 6]
4642-05-25


#Dataset_Generater

In [196]:
def prepare_date_strs(date_strs, tokenizer):
    ids = [dt for dt in tokenizer.texts_to_sequences(date_strs)]
    X = tf.ragged.constant(ids, ragged_rank=1)
    return X.to_tensor() #ragged 텐서를 기본 텐서로 부족한 부분은 0 패딩토큰으로 채운다.

def create_dataset(n_dates):
    X, Y = random_dates(n_dates)
    return prepare_date_strs(X, input_tokenizer), prepare_date_strs(Y, output_tokenizer)

In [197]:
X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [198]:
print(X_train[0], Y_train[0], sep='\n')
print(input_tokenizer.sequences_to_texts([X_train[0].numpy()])[0][::2])
print(output_tokenizer.sequences_to_texts([Y_train[0].numpy()])[0][::2])

tf.Tensor([37  8  7 16  4  1  2 28 18 26 27 28 22 23 22 20  0  0], shape=(18,), dtype=int32)
tf.Tensor([ 6  7  6  4 11  2  1 11  2 10], shape=(10,), dtype=int32)
October 19, 5653
5653-10-19


#Seq2Seq_Model

In [199]:
embedding_size = 32
#출력 최대 길이 : 10 (yyyy-mm-dd)
max_output_length = Y_train.shape[-1]

#인코더 디코더 구조를 사용한다.

#인코더, 시퀀스 투 벡터 구조
encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_vocab_size + 1, output_dim=embedding_size, input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(output_vocab_size + 1, activation='softmax')
])

model = keras.models.Sequential([
    encoder,
    #출력길이가 10이니깐 인코더 출력을 10번 반복해서, 디코더가 10개의 출력을 만들도록함
    keras.layers.RepeatVector(max_output_length),
    decoder
])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_29 (Sequential)  (None, 128)               83680     
                                                                 
 repeat_vector_9 (RepeatVect  (None, 10, 128)          0         
 or)                                                             
                                                                 
 sequential_30 (Sequential)  (None, 10, 12)            133132    
                                                                 
Total params: 216,812
Trainable params: 216,812
Non-trainable params: 0
_________________________________________________________________


In [200]:
optimizer = keras.optimizers.Nadam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [201]:
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 7s 11ms/step - loss: 1.9204 - accuracy: 0.3146 - val_loss: 1.6319 - val_accuracy: 0.4071
Epoch 2/20
313/313 [==============================] - 3s 8ms/step - loss: 1.5181 - accuracy: 0.4451 - val_loss: 1.2753 - val_accuracy: 0.5367
Epoch 3/20
313/313 [==============================] - 3s 8ms/step - loss: 1.1966 - accuracy: 0.5766 - val_loss: 1.3641 - val_accuracy: 0.5108
Epoch 4/20
313/313 [==============================] - 3s 8ms/step - loss: 1.1416 - accuracy: 0.5909 - val_loss: 0.9092 - val_accuracy: 0.6796
Epoch 5/20
313/313 [==============================] - 3s 8ms/step - loss: 0.9386 - accuracy: 0.6624 - val_loss: 0.9684 - val_accuracy: 0.6582
Epoch 6/20
313/313 [==============================] - 3s 8ms/step - loss: 0.8144 - accuracy: 0.7015 - val_loss: 0.6763 - val_accuracy: 0.7409
Epoch 7/20
313/313 [==============================] - 3s 8ms/step - loss: 0.5441 - accuracy: 0.7826 - val_loss: 0.4548 - val_accuracy: 0.8188
Epoch

In [202]:
model.evaluate(X_test,Y_test)

63/63 [==============================] - 0s 4ms/step - loss: 0.0042 - accuracy: 1.0000


[0.004218000452965498, 1.0]

#Model_Return_To_Strings

In [223]:
def ids_to_strings(ids, tokenizer):
    return tokenizer.sequences_to_texts(ids)

print('-'*25)
X_new = X_test[:3]
X_samples = ids_to_strings(X_new.numpy(), input_tokenizer)
print(*[x_sample[::2] for x_sample in X_samples], sep='\n')


print('-'*25)
Y_new = np.argmax(model.predict(X_new), axis=-1)
Y_new = ids_to_strings(Y_new, output_tokenizer)
print(*[y_sample[::2] for y_sample in Y_new], sep='\n')

-------------------------
September 01, 4264
September 22, 5046
October 16, 5080
-------------------------
4264-09-01
5046-09-22
5080-10-16
